# Mini Lab : Logistic Regression and Support Vector Machine

### Group 3 - Members:

_Tai Chowdhury_<br>
_Apurv Mittal_<br>
_Ravi Sivaraman_<br>
_Seemant Srivastava_<br>


## Introduction

As discussed in Lab 1, we have acquired the Australian Weather dataset from Kaggle portal. It contains 10 years of weather data collected from many locations across Australia. These are daily weather observations. There are 145,459 observations with 23 attributes in the original dataset. 

We have chosen RainTomorrow (categorical) and Rainfall (continuous) as predictor variables. RainTomorrow is a categorical attribute which indicates whether it is going to rain tomorrow - yes or no. Rainfall is a continuous attribute that measures amount of rainfall each of the particular locations have received (in mm). Using our models, we will be able to design an algorithm where the bureau can help to predict rainfall for different regions in Australia.

In this Lab 2 assignment, we have measured the accuracy and effectiveness of our model for categorical variable RainTomorrow by using 10-fold cross validation against the confusion matrix measurements like: Precision, Recall and Accuracy. We have explored the methods of logistic regression and support vector machine (SVM) models on our dataset. 

We have used `scikit-learn` packages for our exploration. We ran logistic regression models with all the available solvers in the `scikit-learn` package and compare the effictiveness and accuracy of the model to predict `RainfallTomorrow`. We also measured the duration of model run from each models to compare model performance and efficiency as well.
 
To get started, we will start with loading all the necessary packages for our analysis. We will start our analysis with `df_impute` which is the imputed dataframe from our last explanatory data analysis Lab 1 project. Using this dataframe will ensure data consistency for all the labs going forward.

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from scipy import stats
import warnings
from shapely.geometry import Point
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.model_selection import ShuffleSplit
from sklearn.utils import resample

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn import metrics

In [ ]:
#Ignore Warnings on final

warnings.filterwarnings('ignore')

## Imputed Data
We imputed data in EDA and reusing imputed data from EDA (Lab1) project.
Here is the link to the EDA

https://nbviewer.jupyter.org/github/ravisiv/AussieWeatherEDA/blob/c0ba412cb75da21eba386ea9ea39f645ad6af1d0/DS7331_Lab1_Group3_Ravi_Taifur_Seemant_Apurv_Submission.ipynb


In [ ]:
# Read the Imputed Australia weather data
df_impute = pd.read_csv("weatherAUS_imputed.csv")

In [ ]:
#  View the top rows of the data imported
df_impute.head()

The imputed data doesn't include any null or missing values. Also, we have dropped the columns like: Date of observation and City Name.

In [ ]:
df_impute_num = df_impute.columns[df_impute.dtypes == 'float64']
df_impute_cat=df_impute.columns[df_impute.dtypes == 'object']
print("Numeric Variables:", df_impute_num)
print("Categorical Variables:", df_impute_cat)

Before continuing further, we need to check which variables are numeric and which are not. As the models expect numerical variables. We will filter and identify non-numeric variables.

`WindGustDir`, `WindDir9am`, `WindDir3pm`, `RainToday` and `RainTomorrow`are not numeric. Here `RainTomorrow` is our response variable. we handle the other variables with hot-one-encoding later in the flow.

In [ ]:
#Keep the original data
df_model = df_impute.copy()

Creating a new DataFrame `df_model` for modeling to avoid any changes to the original dataset `df_impute`.

In [ ]:
# Create a new variable to Identify if it RainToday

df_model["IsRainToday"] = df_impute['RainToday']

# Replacing No with 0 and Yes with 1.

df_model['IsRainToday'].replace({'No': 0, 'Yes': 1},inplace = True)


Assigning `0` to No values and `1` to Yes values in `RainToday` (Changed to `IsRainToday`)

In [ ]:
print("df_impute", df_impute.shape)
print("df_model", df_model.shape)


In [ ]:
# Printing the values to check if the data looks good

df_model.head()

We can observe that the presence of “0” and “1” is almost in the 78:22 ratio. So there is a class imbalance and we have to deal with it. To fight against the class imbalance, we will use here the oversampling of the minority class. Since the size of the dataset is quite small, majority class subsampling wouldn’t make much sense here.

## One-hot encoding

Before we create our models, we need to format our attributes. We are converting `RainToday` and `RainTomorrow` into numeric variables to `0` and `1`. We also decided to go ahead with one-hot-encoding `WindGustDir`, `WindDir9am`, and `WindDir3pm` attributes based on the direction of the wind. 

In [ ]:
# perform one-hot encoding using dummies

gust_df = pd.get_dummies(df_model.WindGustDir,prefix='GustDir')
wind3pm_df = pd.get_dummies(df_model.WindDir3pm,prefix='Wind3pm')
wind9am_df = pd.get_dummies(df_model.WindDir9am,prefix='Wind9am')
df_model = pd.concat((df_model,gust_df, wind3pm_df, wind9am_df),axis=1) # add back into the dataframe


We decided to do one-hot-encoding using dummies function as machine learning algorithms and models requires numerical values for both input and output attributes.

In [ ]:
# Drop categorical columns

df_model = df_model.drop(['WindDir3pm', 'WindDir9am', 'WindGustDir', 'RainToday'], axis = 1)

After conversions, we are removing these categorical attributes to avoid duplicates as we have those data in numerical format. We are added the newly formatted attributes and rest of the continuous attributes into a new dataframe - df_model. We will use the new dataframe for modeling.

Reference: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [ ]:
#Check if Yes is replaced as 1

print("Are there 1's and 0's in the RainToday column?", 
      (df_model['IsRainToday'].sum() > 0) and (df_model['IsRainToday'].sum() < len(df_model['IsRainToday'])))

#Non zero output means there is a mixture of 1's and 0's


Checking if the data imputation happened accurately.

In [ ]:
df_model_num = df_model.columns[df_model.dtypes == 'float64']
df_model_cat=df_model.columns[df_model.dtypes == 'object']
print("Numeric Variables:", df_model_num)
print("Categorical Variables:", df_model_cat)

Check if all the numerical variables are accurately created and if we still have any non-numeric data.

In [ ]:
X=df_model[df_model_num]
y = df_model.RainTomorrow
print('features shape:', X.shape) 
print('target shape:', y.shape )

Assigning the `RainTomorrow` as our response variable (y) and all other variables include one-hot-encoded values as X.

### Data Distribution

Check if the data distribution is balanced or not for the response variable `RainTomorrow`.

In [ ]:
fig = plt.figure(figsize = (8,5))
df_model['RainTomorrow'].value_counts(normalize = True).plot(kind='bar', color= ['skyblue','navy'], alpha = 0.9, rot=0)
plt.title('RainTomorrow Indicator No(0) and Yes(1) in the Imbalanced Dataset')
plt.show()

As expected, we see the data for `RainTomorrow` is imbalanced. Majority of the data is for `No` rain vs. `Yes` for `RainTomorrow`.

We can observe that the presence of `0` and `1` is almost in the `78:22` ratio. We will be cognizant of the fact that our model may be not very effective if we don't solve for imbalance. We will discuss and adjust for this imbalance in our analysis.

## Logistic Regression Model

### Creating Logistic Regression Function

The function below creates models with various parameters supported by LogisticRegression. There are various combinations of
1. `Model Type`
2. `Class Weight`
3. `Solver`
4. `C`
5. `Penalty`
6. `Iterations`

This function runs for specific Solver, C, Penalty but takes various iterations, and we used 10 iterations.
When this function called with Solver, C, Penalty, the function runs for BOTH Class Weight, `balanced` and `None`. The result of both `balanced` and `None` is stored as single row in a dataframe.
They are stored in the same row to verify how `balanced` and `None` compares.

##### Model Type

1. `Shuffle`
2. `Stratified`

`ShuffleSplit`  — is similar to `Cross Validation` where we can specify the percentage of split for train and test data. However, in regular cross-validation, the data is not split randomly, so, it is good to shuffle the targets before applying the `cross-validation`.


`Stratified` — CV technique is very useful with unbalanced dataset. As discussed above our dataset is not balanced and rightly so, we don't expect it to rain and no-rain days to be equal in Australia. The data is expected to be unbalanced and expected to be such in future as well. So using stratified sampling techniques gives us the ability to preserve the proportion of the Rain days vs non-rain days in our dataset. We can be confident that the Train and Test split data is not leaving out important information like entire dataset is of `No` rain days which will give highly inaccurate output eventhough the accuracy might be maintained. 

In `Stratified` sampling, the data is k-1 split in favor of Train vs Test data.


References: 
https://towardsdatascience.com/understanding-8-types-of-cross-validation-80c935a4976d
https://mclguide.readthedocs.io/en/latest/sklearn/cv.html


##### Solver Options
Scikit-learn comes with five different solver options. Each solver minimize the cost function. Here are the five options:

`newton-cg` — A newton method. Newton methods use an exact Hessian matrix. It's slow for large datasets, because it computes the second derivatives.

`lbfgs` — Stands for Limited-memory Broyden–Fletcher–Goldfarb–Shanno. It approximates the second derivative matrix updates with gradient evaluations. It stores only the last few updates, so it saves memory. It isn't super fast with large data sets.

`liblinear` — Library for Large Linear Classification. Uses a coordinate descent algorithm. Coordinate descent is based on minimizing a multivariate function by solving univariate optimization problems in a loop. In other words, it moves toward the minimum in one direction at a time. It performs pretty well with high dimensionality. It does have a number of drawbacks. It can get stuck, is unable to run in parallel, and can only solve multi-class logistic regression with one-vs.-rest.

`sag` — Stochastic Average Gradient descent. A variation of gradient descent and incremental aggregated gradient approaches that uses a random sample of previous gradient values. Fast for big datasets.

`saga` — Extension of sag that also allows for L1 regularization. Should generally train faster than sag.

Reference for above definitions: https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451

##### Penalty

Used to specify the norm used in the penalization. The `newton-cg`’, `sag` and `lbfgs` solvers support only `l2` penalties. `elasticnet` is only supported by the `saga` solver. If `none` (not supported by the `liblinear` solver), no regularization is applied.

`L1`  or `Lasso` — (`LASSO` is Least Absolute Shrinkage and Selection Operator) uses a penalized least squares approach that squeezes the regression coefficients to 0 when the penalty is large. The algorithm starts with a large penalty and gradually relaxes the penalty to allow for a single variable to be added into the model (the coefficient is no longer `0`). `LASSO` uses `L1` method.

`L2` or `Ridge` — adds a penalty equal to the square of the magnitude of coefficients. `L2` will not yield sparse models and all coefficients are shrunk by the same factor (not eliminated). Ridge regression and SVMs use this method.

`elasticnet`  — Procedure identical to `LASSO` however the penalty is different. `elasticnet` uses a combination of both the `LASSO` penalty as well as the `RIDGE` regression penalty. It combine both `L1` & `L2` methods.

`none` — No penalty

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Reference for Penalty definitions: https://www.statisticshowto.com/regularization/

##### C
`C` is cost, which we run with `1.0`, `10.0` and `100.0` for each combination of Solver and Penalty.

If the solver doesn't support the penalty, those are skipped.


The dataframe stores:

1. `Model Type`
2. `Solver`
3. `C`
4. `Penalty`
5. `Iteration`
6. `AccuracyNone`
7. `MacroAvgPrecisionNone`
7. `WeightedAvgPrecisionNone`
8. `DiffMacro/WeightedNone`
9. `fprNone`
10. `tprNone`,
11. `AccuracyBalanced`
12. `MacroAvgPrecisionBalanced`
13. `WeightedAvgPrecisionBalanced`
14. `DiffMacro/WeightedBalanced`
15. `fprBalanced`
16. `tprBalanced`
17. `Classes`

`DiffMacro/WeightedBalanced` is the difference between the Presicion of `macro avg` and `Weighted`. Smaller the value, more closer they are.

`fprNone` and `tprNone` are stored to run ROC curve.

`Classes` is `Yes` and `No` as an array, which are used for labels for plots. They are constant for all rows.



In [ ]:
#Logistic Regression Model

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from timeit import default_timer as timer

#model stats dataframe columns for class weight balanced and None
model_stats_columns=["Model","Solver", "C", "Penalty","Iteration",
                         "AccuracyNone", 
                         "MacroAvgPrecisionNone","MacroAvgRecallNone", "MacroAvgf1-scoreNone",
                         "WeightedAvgPrecisionNone", "WeightedAvgRecallNone", "WeightedAvgf1-scoreNone",
                         "DiffMacro/WeightedNone","fprNone", "tprNone","lr_clfNone",
                         "AccuracyBalanced",
                         "MacroAvgPrecisionBalanced","MacroAvgRecallBalanced", "MacroAvgf1-scoreBalanced",
                         "WeightedAvgPrecisionBalanced", "WeightedAvgRecallBalanced", "WeightedAvgf1-scoreBalanced",
                         "DiffMacro/WeightedBalanced","fprBalanced", "tprBalanced", "lr_clfBalanced",
                         "Classes", "Time"
                        ]


def create_log_models(model_type,df, iterations,penalty, C, solver):
    class_weight = ['balanced', None]
    
    #Create logreg object for both class weights
    lr_clf_balanced = LogisticRegression(penalty=penalty, C=C, class_weight='balanced', solver=solver) 
    lr_clf_none = LogisticRegression(penalty=penalty, C=C, class_weight=None, solver=solver)
    
    #Store both objects in a dict for later retrival
    lr_clf_dict = { 
                    "balanced": lr_clf_balanced,
                    "None": lr_clf_none
                    }

    num_cv_iterations = iterations

   
    if "RainTomorrow" in df:
        y = df["RainTomorrow"].values # get the labels we want
        del df["RainTomorrow"] # get rid of the class label
        X = df.values # use everything else to predict!
    else:
        return

    num_instances = len(y)
    
    cv_data = None
    if model_type == "shuffle":
        cv_data = ShuffleSplit(n_splits=num_cv_iterations,
                             test_size  = 0.2, random_state = 123)

    elif model_type == "stratified":
        cv_data = StratifiedKFold(n_splits=iterations, random_state=123, shuffle=True)
        cv_data.get_n_splits(X, y)
       
    #Initialize variables
    iter_num=0
    rows = []
    stats_dict = {}
    target_names = ['No', 'Yes']
    classes = None    
    scl_obj = StandardScaler()
    lr_clf = None
    # Run for balanced first with same model and then None with same model
    # store the results in same row of dataframe
    # This helps to compare none and balanced macro avg
   
    for train_indices, test_indices in cv_data.split(X,y): 
        starttime = timer()
        for cw in class_weight:
            X_train = X[train_indices]
            y_train = y[train_indices]
        
            scl_obj.fit(X_train)

            X_test = X[test_indices]
            y_test = y[test_indices]
            
            #Get the logistic regression object for the current cw class weight
            key = None
            if cw == None:
                key = "None"
            else:
                key = cw
                    
            lr_clf = lr_clf_dict[key]
            
            try:
                X_train_scaled = scl_obj.transform(X_train) 
                X_test_scaled = scl_obj.transform(X_test)

                lr_clf.fit(X_train_scaled,y_train)  # train object
                y_hat = lr_clf.predict(X_test_scaled) # get test set precitions
                classes = lr_clf.classes_

                acc = mt.accuracy_score(y_test,y_hat)
                conf = mt.confusion_matrix(y_test,y_hat)
                
                class_report = classification_report(y_test, y_hat, target_names, output_dict=True)
                
                # Macro avg stats
                macro_avg_precision = class_report["macro avg"]["precision"]
                macro_avg_recall = class_report["macro avg"]["recall"]
                macro_avg_f1_score = class_report["macro avg"]["f1-score"]

                #Weighted avg stats
                weighted_avg_precision = class_report["weighted avg"]["precision"]
                weighted_avg_recall = class_report["weighted avg"]["recall"]
                weighted_avg_f1_score = class_report["weighted avg"]["f1-score"]

                # Create ROC Curve
                y_test_01 = np.where(y_test =="Yes", 1, [0])
                y_hat_01 = np.where(y_hat =="Yes", 1, [0])

                fpr, tpr, threshold = metrics.roc_curve(y_test_01, y_hat_01)
                
                #Create a dict of these stats for class weight
                #dict will contain stats for balanced on one run, None for the next run
                
                stats_dict[key] = [acc, 
                          macro_avg_precision, macro_avg_recall, macro_avg_f1_score,
                          weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                          abs(weighted_avg_precision-macro_avg_precision),         
                          fpr, tpr, lr_clf ]

                print(model_type, solver,cw,C,penalty,iter_num,"✅")
            except Exception as e:
                #print('Error:', str(e))
                raise
            #end try block  
        #end first for loop
        #When cursor comes here, model has ran for both None and balanced
        #Create a single row of lists combining none and balanced 
        endtime = timer()
        time_taken = endtime - starttime
        row = [model_type, solver,C,penalty,iter_num] + stats_dict["None"] + stats_dict['balanced'] + [classes, time_taken ]
        rows.append(row)
        iter_num+=1
    #end next for loop
    
    #Create a dataframe with the model stats 
    df_ret = pd.DataFrame(rows, columns = model_stats_columns)
    return df_ret


### Running Logistic Regression with various options

The code below runs all the combinations of `Penalty`, `Solver`, `C` and shuffle types and stores in dataframe for further analysis later on.

If any combinations of `Solver` and `Penalty` isn't supported by Logistic Regression, they are skipped.

`iteration` — 10 iterations


In [ ]:


penalty=['l1','l2', 'elasticnet', 'none']
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
model_type = ['shuffle', 'stratified']
C=[1.0, 10.0, 100.0]
iterations = 10

#penalty=['l2']
#solver = ['liblinear']
#model_type = ["stratified"]
#C=[1.0]
#iterations = 2

model_perf_df = pd.DataFrame(columns= model_stats_columns)

# Run all combinations of penalty, solver, model_type, C
# Create a giant dataframe
# Each row of dataframe contains stats for each combination of penalty, solver, model_type,C
# same row contains the stats for both None and balanced class weight
# so we can compare the balanced and None
# and pick the model whose diff in precision (for macro avg) is lowest with highest accuracy
# This model is the closest to real world

for pen in penalty:
    for c_index in C:
        for solv in solver:
            for mdl_type in model_type:
                try:

                    df_ret = create_log_models(model_type=mdl_type,df=df_model,iterations=iterations, penalty=pen, C=c_index, solver=solv)
                    
                    model_perf_df = model_perf_df.append(df_ret, ignore_index=True)
                    
                    #Model deletes RainTomorrow from dataframe
                    #put it back from imputed data to run for another set of model
                    df_model["RainTomorrow"] = df_impute["RainTomorrow"].values
                except Exception as e:
                    #print("Error in running", str(e))
                    continue
                            

                        

In [ ]:
perf_model_copy = model_perf_df.copy()
#model_perf_df = perf_model_copy.copy()


In these models we used two different methodologies for cross validation namely `Shuffle` and `Stratified`.

`ShuffleSplit`  — is similar to `Cross Validation` where we can specify the percentage of split for train and test data. However, in regular cross-validation, the data is not split randomly, so, it is good to shuffle the targets before applying the `cross-validation`.


`Stratified` — CV technique is very useful with unbalanced dataset. As discussed above our dataset is not balanced and rightly so, we don't expect it to rain and no-rain days to be equal in Australia. The data is expected to be unbalanced and expected to be such in future as well. So using stratified sampling techniques gives us the ability to preserve the proportion of the Rain days vs non-rain days in our dataset. We can be confident that the Train and Test split data is not leaving out important information like entire dataset is of `No` rain days which will give highly inaccurate output eventhough the accuracy might be maintained. 

In `Stratified` sampling, the data is k-1 split in favor of Train vs Test data.


References: 
https://towardsdatascience.com/understanding-8-types-of-cross-validation-80c935a4976d
https://mclguide.readthedocs.io/en/latest/sklearn/cv.html


After running the big `Logistic Regression` model for several combinations. We got output for 660 models.

To compare the outputs and make it easier to identify the most appropriate model in terms of accuracy, precision and other factors applicable for Machine Learning models. We decided to put a list of subset of variables including:

`Model` — specifies which technique was used for the logistic model. In this case its between ShuffleSplit and Stratified.

`AccuracyNone` —  specifies the Accuracy observed by the model where data was not `balanced`.

`DiffMacro/WeightedNone` — takes the `Macro Average` and `Weighted Average` of `Precision` from the classification matrix and calculates the difference. The reason to calculate the difference is to check how much variation is in the Precision values based on how the data is split. More details about Precision and Averages is provided below. This variation is calculated on the non `balanced` data.

`AccuracyBalanced` — specifies the Accuracy observed by the model where data was `balanced`.

`DiffMacro/WeightedBalanced` — same as above this variable calculates the difference between Macro and Weighted Average of Precision of a `balanced` data.

### Important Terms

`Accuracy` — is a ratio of correctly predicted observation to the total observations. It is a very important aspect to define the success of a model but just the measurement on its own can be deceiving if the observations are not equal for each class. In such cases we might be predicting accurately for one particular class with large observation and may not do very well for other classes.

`Precision` — is the ratio of correctly predicted positive observations to the total predicted positive observations. Precision = TP/TP+FP  

`Weighted Average` — can be calculated on various output variables of the classification report like Precision, Recall, f1-score. As the name suggests it gives the weighted average of the parameter based on the number of observations or values for each class.

`Macro Average` — similar to weighted average, macro average can also be calculated on various output variables of the classification report like Precision, Recall, f1-score. However, the similarity ends here as unlike weighted average, we don't use weights based on the number of observations, rather equal weights are given to each class to calcualte the value. This tells us if the Precision is as good if the dataset was balanced.

References:
https://blog.exsilio.com/all/accuracy-precision-recall-f1-score-interpretation-of-performance-measures/
https://datascience.stackexchange.com/questions/65839/macro-average-and-weighted-average-meaning-in-classification-report

In [ ]:
from  matplotlib import pyplot
import seaborn

def draw_scatter(df, x,y, title=None):

    seaborn.set(style='ticks')
    cv_type = ['shuffle', 'stratified']

    #fg = seaborn.FacetGrid(data=model_perf_df, hue='Model', hue_order=cv_type, height=8, aspect=1.61, ylim=(0,1))
    fg = seaborn.FacetGrid(data=model_perf_df, hue='Model', hue_order=cv_type, height=8, aspect=1.61)
    fg.map(pyplot.scatter, x, y ).add_legend()
    #fg.suptitle(title)

    for i, ax in enumerate(fg.fig.axes):   
         ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)


#Reference: https://stackoverflow.com/questions/14885895/color-by-column-values-in-matplotlib
#Ref for Axis rotation: https://stackoverflow.com/questions/26540035/rotate-label-text-in-seaborn-factorplot/43256409#43256409

### Compare Shuffle vs Stratified

To compare `Shuffle` and `Stratified`, we plot Shuffle and Stratified with `AccuracyNone` (accuracy for non-balanced data) and `AccuracyBalanced` (accuracy for balanced data).

In the plots below, we see the difference between `Shuffle` and `Stratified` in absolute values is very small

1. `AccuracyNone` values range from `0.871` to `0.877`, which is `0.006`, is a very small difference
2. `AccuraceBalanced` values range from `0.847` to `0.853`, which is `0.006`, is also very small difference

Based on the Accuracy alone, we cannot pick any models from one over other. 
We have to do further analysis of better model, as all models are very close to each other for our dataset.


In [ ]:

#Create a plot by looking shuffle vs stratefied

model_perf_df['Models'] = model_perf_df['Model'] + " / Solver:" + model_perf_df['Solver'] + " / Penalty:" + model_perf_df['Penalty'] + " / C:"+model_perf_df['C'].astype(str)

model_perf_df = model_perf_df.sort_values(by=['Models'],ascending=False)

#Sort by accuracy and take top 5

#model_perf_df = model_perf_df.nlargest(5, 'AccuracyNone')

draw_scatter(df=model_perf_df, x='Models', y='AccuracyNone')


In [ ]:

#Create a plot by looking shuffle vs stratefied

model_perf_df['Models'] = model_perf_df['Model'] + " / Solver:" + model_perf_df['Solver'] + " / Penalty:" + model_perf_df['Penalty'] + " / C:"+model_perf_df['C'].astype(str)

model_perf_df = model_perf_df.sort_values(by=['Models'],ascending=False)

#Sort by accuracy and take top 5

#model_perf_df = model_perf_df.nlargest(5, 'AccuracyNone')

draw_scatter(df=model_perf_df, x='Models', y='AccuracyBalanced')



### Macro Avg vs Weight Avg Precision comparisons

As we couldn't pick the models just from `Accuracy`, we are further our analysis by plotting the difference between the `Macro Avg` precision and `Weighted Avg` precision for all the models. 


   `DiffMacro/WeightedNone` — takes the `Macro Average` and `Weighted Average` of `Precision` from the classification matrix and calculates the difference. The reason to calculate the difference is to check how much variation is in the `Precision` values based on how the data is split. 

   `DiffMacro/WeightedBalanced` — same as above this variable calculates the difference between Macro and Weighted Average of Precision of a `balanced` data.
   
The smaller the difference between `Weight Avg` and `Macro Avg`, the model is closer to real world and also consistent.   



### Weighted Avg vs Macro Avg Precision Plots

In [ ]:

cv_type = ['shuffle', 'stratified']
fg = seaborn.FacetGrid(data=model_perf_df, hue='Model', hue_order=cv_type, height=8, aspect=1.61)    
fg.map(pyplot.scatter, 'Models', 'DiffMacro/WeightedBalanced').add_legend()
#fg.suptitle('Macro Avg - Weighted Avg Precision of Balanced Class Weight')

for i, ax in enumerate(fg.fig.axes):   ## getting all axes of the fig object
     ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)

#Reference: https://stackoverflow.com/questions/14885895/color-by-column-values-in-matplotlib

#Ref for Axis rotation: https://stackoverflow.com/questions/26540035/rotate-label-text-in-seaborn-factorplot/43256409#43256409

Now, we plotted the models classified into `Shuffle` and `Stratified` against `DiffMacro/WeightedBalanced` of precision (difference between Macro and Weighted Average of Precision of a balanced data).

In [ ]:
cv_type = ['shuffle', 'stratified']
fg = seaborn.FacetGrid(data=model_perf_df, hue='Model', hue_order=cv_type, height=8, aspect=1.61)    
fg.map(pyplot.scatter, 'Models', 'DiffMacro/WeightedNone').add_legend()
#fg.suptitle('Macro Avg - Weighted Avg Precision of None Class Weight')
for i, ax in enumerate(fg.fig.axes):   ## getting all axes of the fig object
     ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)



Also, plotted the models classified into `Shuffle` and `Stratified` against `DiffMacro/WeightedNone` of precision (difference between Macro and Weighted Average of Precision of non-balanced data).

From the above two plots, we notice that the difference between Macro Average of Precision and Weighted Average of Precision varies as below:

Balanced Data - `0.091` to `0.097`

non-Balanced Data - `0.037` to `0.042`

There are two major takeaways from the above plots. 

1. The `stratifiedkfolds` data sees lower variation compared to `shufflesplit`.

2. Its evident that the difference between Macro Average of Precision and Weighted Average of Precision varies a lot more for `Balanced` data compared to `non-balanced` data. 

Since, lower variation is desired to get more stable and consistent model. We decide to go ahead with `Stratified` and non-balanced data. In our model we have hypertuned `shuffle` as `True` for `stratifiedkfolds` as well. Also, we know `stratifiedkfolds` works well with non-balanced data.

With above points in mind, we will continue with `Stratified` and `non-balanced` data models.

## Model cleanup

We have established in the previous sections that `StratifiedKFold` works better for our dataset.

The below code removes `ShuffleSplit` model from further analysis

In [ ]:
#Lets focus on Stratified

#Remove all shuffle

model_perf_df.drop(model_perf_df[model_perf_df.Model == "shuffle"].index, inplace=True)

In [ ]:
model_perf_df

From the previous sections we have established, we are removing `balanced` `Class Weight` from our further analysis

In [ ]:
# Remove Columns of balanced

balanced_columns=["AccuracyBalanced",
                         "MacroAvgPrecisionBalanced","MacroAvgRecallBalanced", "MacroAvgf1-scoreBalanced",
                         "WeightedAvgPrecisionBalanced", "WeightedAvgRecallBalanced", "WeightedAvgf1-scoreBalanced",
                         "DiffMacro/WeightedBalanced","fprBalanced", "tprBalanced" 
                        ]

model_perf_df = model_perf_df.drop(balanced_columns, axis=1)


`model_perf_df` has following model data:

1. Stratified
2. non-balanced Class Weight



In [ ]:
model_perf_df

## Pick top model

As `Accuracy` and difference between `Weighted Precision` and `Macro Avg Precision` is not significantly varies across the models, we shall sort by `Recall` as for our data, `RainTomorrow` as `Yes` prediction is more valuable. 

`Recall` is the number of true positives (`TP`) divided by the number of true positives plus the number of false negatives (`FN`). 

`Recall` = TP/(TP + FN)


In [ ]:
model_perf_df = model_perf_df.sort_values(by=['MacroAvgRecallNone','AccuracyNone', 'MacroAvgf1-scoreNone'], ascending=False)

# Pick Top 5

model_perf_df.head(5)

In [ ]:
top_model = model_perf_df.head(1)

Below shows our top logistic regression model.

In [ ]:
top_model

### ROC Curve

Now we will plot the ROC curve for our selected top model. 

`AUC` - `ROC` curve is a performance measurement for the classification problems at various threshold settings. `ROC` is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes. Higher the `AUC`, the better the model is at predicting 0s as 0s and 1s as 1s. 

reference: https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5

In [ ]:
#ROC Curve of top first model

for index, row in top_model.iterrows():
    fpr = row["fprNone"]
    tpr = row["tprNone"]
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## SVM Model

The function below creates a `SVM` model with the given parameters.

The function accepts following paramters:

1. iterations
2. kernel
3. degree
4. gamma


We are using StratifiedKFold because ShuffleSplit (with 80/20 split) is not as good as Stratified for our data. We have explained the rationale for using Stratified in the Logistic Regression section [above](# Look-only-Stratified-models).

We are using `Class Weight` as `None`, as `StratifiedKFold` works well for our data; so we decided not to use `Class Weight` as balanced.

We are only running the certain combinations we found useful for our data from Logistic Regression, as we are constrianed by time, as running all combinations of parameters for SVM takes a long time to complete.

`gamma` — is run only with `auto`, as our data is scaled already. We decided not to use `scale` option.

`iteration` — 10 iterations


In [ ]:



def create_svm_model(df, C, iterations):
    num_cv_iterations = iterations

    model_stats_columns=["Model","Solver", "C", "Penalty","Iteration",
                         "AccuracyNone", 
                         "MacroAvgPrecisionNone","MacroAvgRecallNone", "MacroAvgf1-scoreNone",
                         "WeightedAvgPrecisionNone", "WeightedAvgRecallNone", "WeightedAvgf1-scoreNone",
                         "DiffMacro/WeightedNone","fprNone", "tprNone",
                         "Classes"
                        ]


    
    if "RainTomorrow" in df:
        y = df["RainTomorrow"].values # get the labels we want
        del df["RainTomorrow"] # get rid of the class label
        X = df.values # use everything else to predict!
    else:
        return

    num_instances = len(y)
    
    # ShuffleSplit is not a good representation of our data. We already established in the previous
    # models that Stratified is better for our model.
    # We are choosing to run only Stratified for SVM
    
    cv_data = StratifiedKFold(n_splits=iterations, random_state=123, shuffle=True)
    cv_data.get_n_splits(X, y)

    iter_num=0
    rows = []
    scl_obj = StandardScaler()
    model_type = "svm"
    kernel = "rbf"
    penalty = None
    cw = None
    stats_dict = {}
    solver = None
    classes = None
    
    
    for train_indices, test_indices in cv_data.split(X,y): 
        X_train = X[train_indices]
        y_train = y[train_indices]
        
        scl_obj.fit(X_train)

        X_test = X[test_indices]
        y_test = y[test_indices]
        try:
            X_train_scaled = scl_obj.transform(X_train) 
            X_test_scaled = scl_obj.transform(X_test)
            y_hat = None
          
            
            svm_clf = SVC(C=C, kernel=kernel, degree=3, gamma='auto') 
            svm_clf.fit(X_train_scaled, y_train)  # train object
            y_hat = svm_clf.predict(X_test_scaled) # get test set precitions
            classes = svm_clf.classes_

            acc = mt.accuracy_score(y_test,y_hat)
            conf = mt.confusion_matrix(y_test,y_hat)
            target_names = ['No', 'Yes']
            class_report = classification_report(y_test, y_hat, target_names, output_dict=True)
            
            # Macro avg stats
            macro_avg_precision = class_report["macro avg"]["precision"]
            macro_avg_recall = class_report["macro avg"]["recall"]
            macro_avg_f1_score = class_report["macro avg"]["f1-score"]
            
            #Weighted avg stats
            weighted_avg_precision = class_report["weighted avg"]["precision"]
            weighted_avg_recall = class_report["weighted avg"]["recall"]
            weighted_avg_f1_score = class_report["weighted avg"]["f1-score"]
            
            
            # Create ROC Curve
            y_test_01 = np.where(y_test =="Yes", 1, [0])
            y_hat_01 = np.where(y_hat =="Yes", 1, [0])

            fpr, tpr, threshold = metrics.roc_curve(y_test_01, y_hat_01)
            stats_dict["None"] = [acc, 
                          macro_avg_precision, macro_avg_recall, macro_avg_f1_score,
                          weighted_avg_precision,weighted_avg_recall,weighted_avg_f1_score,
                          abs(weighted_avg_precision-macro_avg_precision),         
                          fpr, tpr ]
            
            print(model_type, kernel,cw,C,penalty,iter_num,"✅")
        except Exception as e:
            print('Error:', str(e))
            raise
        
        #we are not running balanced class weight, so the values are set to None
        row = [model_type, solver,C,penalty,iter_num] + stats_dict["None"]  + [classes]
        rows.append(row)
        iter_num+=1
    #end next for loop
    
    #Create a dataframe with the model stats 
    df_ret = pd.DataFrame(rows, columns = model_stats_columns) 
    return df_ret


In [ ]:
%%time
C=[1.0, 10.0, 100.0]

model_stats_columns=["Model","Solver", "C", "Penalty","Iteration",
                         "AccuracyNone", 
                         "MacroAvgPrecisionNone","MacroAvgRecallNone", "MacroAvgf1-scoreNone",
                         "WeightedAvgPrecisionNone", "WeightedAvgRecallNone", "WeightedAvgf1-scoreNone",
                         "DiffMacro/WeightedNone","fprNone", "tprNone",
                         "Classes"
                        ]

#C=[1.0]
iterations = 10

model_perf_svm_df = pd.DataFrame(columns= model_stats_columns)

for c_i in C:
    df_ret_svm = create_svm_model(df=df_model, C=c_i ,iterations=iterations)
    model_perf_svm_df = model_perf_svm_df.append(df_ret_svm)


In [ ]:
model_perf_svm_df.sort_values(by=['AccuracyNone'],ascending=False).head(5)


In [ ]:
from  matplotlib import pyplot
import seaborn
fg = seaborn.FacetGrid(data=model_perf_svm_df, hue='Model', height=8, aspect=1.61)    
fg.map(pyplot.scatter, 'Model', 'AccuracyNone').add_legend()
fg.fig.suptitle('Model By Accuracy')

for i, ax in enumerate(fg.fig.axes):   ## getting all axes of the fig object
     ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)

In [ ]:
from  matplotlib import pyplot
import seaborn
fg = seaborn.FacetGrid(data=model_perf_svm_df, hue='Model', height=8, aspect=1.61)    
fg.map(pyplot.scatter, 'Model', 'MacroAvgPrecisionNone').add_legend()
fg.fig.suptitle('Model By Precision')

for i, ax in enumerate(fg.fig.axes):   ## getting all axes of the fig object
     ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)

### Top SVM Model

We are picking the top model by sorting using `Accuracy`, `F1 score`, and `Recall` scores. 

In [ ]:
model_perf_svm_df = model_perf_svm_df.sort_values(by=['AccuracyNone', 'MacroAvgf1-scoreNone','MacroAvgRecallNone'], ascending=False)


In [ ]:
top_svm_model = model_perf_svm_df.head(1)

In [ ]:
top_svm_model

In [ ]:
# ROC
for index, row in top_svm_model.iterrows():
    fpr = row["fprNone"]
    tpr = row["tprNone"]
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## Feature Importance

In [ ]:
# now let's make a pandas Series with the names and values, and plot them

lr_clf = top_model["lr_clfNone"]
#weights = pd.Series(lr_clf,index=df_model.columns)
#weights.plot(kind='bar', figsize=(14,6))
lr_clf.coef_


In [ ]:
print(df_ret)

In [ ]:
df_model

In [ ]:
# Looking at feature importance 

import shap # SHAP for Explaining Models
shap.initjs()
# Create a tree explainer and understanding the values we have 
shap_ex = shap.LinearExplainer(lr_clf, X_test)
vals = shap_ex.shap_values(X_test)
shap.summary_plot(vals, df_model.columns, plot_type="bar")

# Reference: https://shap.readthedocs.io/en/latest/

In [ ]:
# Summarize the effect of all the features through SHAP summary plot:
explainer = shap.Explainer(lr_clf, X_train, feature_names=df_model.columns)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values)

# Reference: https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/linear_models/Sentiment%20Analysis%20with%20Logistic%20Regression.html

In [ ]:
pip install InvalidSchemeCombination

Explaining the logistic model for Rain Tomorrow prediction:

SHAP summary plots give us a birds-eye view of feature importance and what is driving it. The values of the features are their TF-IDF values.

The above SHAP summary plot is made of many dots. Each dot has three characteristics:

Vertical location shows what feature it is depicting
Color shows whether that feature was high or low for that row of the dataset.
Horizontal location shows whether the effect of that value caused a higher or lower prediction.
For example, the point in the upper left was for 'Sunshine' that caused lower chances of Rain Tomorrow, reducing the prediction by 2.

Some things the summary plot is  able to easily pick out:-

Remember that higher means more likely to be negative, so in the plots above the “red” features are actually helping raise the chance for Rain Tomorrow, while the negative features are lowering the chance for Rain Tomorrow.
The model ignored around 56 features which were of lower importance in predicting the chances of Rain Tomorrow.
Usually 'WindGustSpeed' has moderate effert on the prediction, but there are extreme cases of 'WindGustSpeed' where a high value still caused moderate level of prediction.
High values of Goal scored caused higher predictions, and low values caused low predictions

Reference: https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values

In [ ]:
# Explaining why a sample weather record # 200 is classified as Rain Tomorrow (Yes/No)?
ind = 200
shap.plots.force(shap_values[ind])
# Reference: https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/linear_models/Sentiment%20Analysis%20with%20Logistic%20Regression.html

In [ ]:
# Heatmap plot provides another global view of the model’s behavior, with a focus on predictions for Rain Tomorrow.
#shap.plots.heatmap(shap_values)

In [ ]:
groups_by_time = {
    '3pm': [f for f in df_impute if '3pm' in f],
    '9am': [f for f in df_impute if '9am' in f],
    'not_time_based': [f for f in df_impute if '9am' not in f and '3pm' not in f]
}

groups_by_type = {
    'humidity_and_rain': ['Rainfall',
                          'Evaporation',
                          'Humidity9am',
                          'Humidity3pm',
                          'RainToday'],
    'temperature': ['MinTemp',
                    'MaxTemp',
                    'Temp9am',
                    'Temp3pm'],
    'sun_and_clouds': ['Cloud9am',
                       'Cloud3pm',
                       'Sunshine'],
    'wind_and_pressure': ['WindGustDir',
                          'WindGustSpeed',
                          'WindDir9am',
                          'WindDir3pm',
                          'WindSpeed9am',
                          'WindSpeed3pm',
                          'Pressure9am',
                          'Pressure3pm'],
    'location': ['Location']
}

shap_time = grouped_shap(shap_vals, df_impute, groups_by_time)
shap_type = grouped_shap(shap_vals, df_impute, groups_by_type)



## Alternative/Shorter version of logistic regression

In [ ]:
# and here is an even shorter way of getting the accuracies for each training and test set

accuracies = logistic_regression_short() # this also can help with parallelism
print(accuracies)

In [ ]:
disp = mt.ConfusionMatrixDisplay(confusion_matrix=conf[1])
disp.plot()

In [ ]:
# this does the exact same thing as the above block of code, but with shorter syntax

for iter_num,(train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    lr_clf.fit([train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict([test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

In [ ]:
#Logistic Regression Function
#Run with various parameters
from sklearn.model_selection import cross_val_score

def logistic_regression_short(df, iterations,class_weight, penalty, solver, C, l1_ratio):
    lr_clf = LogisticRegression(penalty=penalty, C=C, class_weight=class_weight, solver=solver) 

  
    if "RainTomorrow" in df:
        y = df["RainTomorrow"].values # get the labels we want
        del df["RainTomorrow"] # get rid of the class label
        X = df.values # use everything else to predict!
    else:
        return

    rows = []
    iter_num = 0
    
    try:
        # train the reusable logisitc regression model on the training data
        accuracies = cross_val_score(lr_clf, X, y=y, cv=iterations) 
        for index, acc in enumerate(accuracies):
            rows.append([solver,class_weight,C,penalty,index, acc, None])
    except Exception as e:
        print("Error in running", solver,class_weight,C,penalty,iter_num, str(e))
        
        
    df_ret = pd.DataFrame(rows, columns=["Solver", "ClassWeight", "C", "Penalty","Iteration","Accuracy","ConfusionMatrix"])
    return df_ret